In [8]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

In [3]:
load_dotenv()
KEY = os.getenv('OPENAI_API_KEY')

In [7]:
llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=.7)

C:\Users\Adeeb\AppData\Local\Temp\ipykernel_11524\2478737479.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=.7)


In [19]:
file_path = './data.txt'
TEXT = ''
with open(file_path, 'r') as file:
    TEXT = file.read()

JSON_FORMAT = {
    "1": {
        "mcq": "Ask the first question",
        "options": {
            "a": 'option 1',
            "b": 'option 2',
            "c": 'option 3',
            "d": 'option 4'
        },
        "correct_answer": "a"
    },
    "2": {
        "mcq": "Ask the second question",
        "options": {
            "a": 'option 1',
            "b": 'option 2',
            "c": 'option 3',
            "d": 'option 4'
        },
        "correct_answer": "b"
    },
    "3": {
        "mcq": "Ask the Third question",
        "options": {
            "a": 'option 1',
            "b": 'option 2',
            "c": 'option 3',
            "d": 'option 4'
        },
        "correct_answer": "d"
    }
}

NUMBER = 5
SUBJECT = 'Machine Learning'
TONE = 'Hard'
json_format_string = json.dumps(JSON_FORMAT)

quiz_generation_template = '''You are a quiz master bot. You are provided with a Input Text. 
You need to generate {number} question quiz based on the provided text for {subject} students. You need to 
set the difficulty level to {tone} level. The output should be in JSON Format as specified at the end of the prompt 

INPUT TEXT: {text}

Use the above input text to generate quiz and produce the output in JSON Format as shown below.

### OUTPUT JSON Format: 
{json_format}
'''
prompt_template = PromptTemplate(
    input_variables=['number','subject', 'tone', 'text', 'json_format'],
    template=quiz_generation_template
)

# client = OpenAI(model_name='gpt-3.5-turbo', openai_api_key=KEY)
quiz_chain = LLMChain(llm=llm, prompt=prompt_template)

response = quiz_chain.invoke({
    'number': NUMBER,
    'subject': SUBJECT,
    'tone': TONE,
    'text': TEXT,
    'json_format': json_format_string
})


In [28]:
mcq_json = json.loads(response['text'])
mcq_json

{'1': {'mcq': 'Who coined the term machine learning in 1959?',
  'options': {'a': 'Donald Hebb',
   'b': 'Walter Pitts',
   'c': 'Tom M. Mitchell',
   'd': 'Arthur Samuel'},
  'correct_answer': 'd'},
 '2': {'mcq': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'options': {'a': 'Cybernet',
   'b': 'Cybertron',
   'c': 'Cyberware',
   'd': 'Cybermind'},
  'correct_answer': 'b'},
 '3': {'mcq': 'According to Tom M. Mitchell, what is the definition of machine learning algorithms?',
  'options': {'a': 'Learning from experience E with respect to tasks T and performance measure P',
   'b': 'Learning from experience T with respect to tasks E and performance measure P',
   'c': 'Learning from experience P with respect to tasks T and performance measure E',
   'd': 'Learning from experience E with respect to performance measure P and tasks T'},
  'correct_answer': 'a'},
 '4': {'mcq': 'What is one of the objectives of modern-day mach

In [46]:
questions = []
for index, item in mcq_json.items():
    mcq = item['mcq']
    
    options = ' | '.join(
        [f"{option}: {option_value}" for option, option_value in item['options'].items()]
    )
    correct_answer = item['correct_answer']
    questions.append({
        'mcq': mcq,
        'options': options,
        'answer': correct_answer
    })
questions


[{'mcq': 'Who coined the term machine learning in 1959?',
  'options': 'a: Donald Hebb | b: Walter Pitts | c: Tom M. Mitchell | d: Arthur Samuel',
  'answer': 'd'},
 {'mcq': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'options': 'a: Cybernet | b: Cybertron | c: Cyberware | d: Cybermind',
  'answer': 'b'},
 {'mcq': 'According to Tom M. Mitchell, what is the definition of machine learning algorithms?',
  'options': 'a: Learning from experience E with respect to tasks T and performance measure P | b: Learning from experience T with respect to tasks E and performance measure P | c: Learning from experience P with respect to tasks T and performance measure E | d: Learning from experience E with respect to performance measure P and tasks T',
  'answer': 'a'},
 {'mcq': 'What is one of the objectives of modern-day machine learning?',
  'options': 'a: To create new models | b: To make predictions for future outcomes | c: To ana

In [49]:
questions_df = pd.DataFrame(questions)
questions_df.head()


,mcq,options,answer
0,Who coined the term machine learning in 1959?,a: Donald Hebb | b: Walter Pitts | c: Tom M. M...,d
1,What was the name of the experimental 'learnin...,a: Cybernet | b: Cybertron | c: Cyberware | d:...,b
2,"According to Tom M. Mitchell, what is the defi...",a: Learning from experience E with respect to ...,a
3,What is one of the objectives of modern-day ma...,a: To create new models | b: To make predictio...,b
4,What is the purpose of a hypothetical algorith...,a: To predict future outcomes | b: To analyze ...,d


In [ ]:
questions_df.to_csv('./quiz.csv')